<a href="https://colab.research.google.com/github/AshishKhatiwada/Account/blob/master/colabs/intro/Intro_to_Weights_%26_Biases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `Llaama3` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Llaama3`


In [1]:
# Install vLLM from pip:
!pip install vllm

In [2]:
# Load and run the model:
!vllm serve "facebook/layerskip-llama3-8B"

INFO 04-28 08:25:25 [__init__.py:239] Automatically detected platform cuda.
2025-04-28 08:25:25.469542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745828725.489494    2854 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745828725.495432    2854 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-28 08:25:25.515332: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO 04-28 08:25:29 [api_server.py:1034] vLLM API

In [14]:
# Install necessary libraries
!pip install transformers datasets huggingface_hub

# Import required libraries
from google.colab import files
import pandas as pd
from datasets import Dataset
from transformers import LlamaTokenizer, LlamaForCausalLM, Trainer, TrainingArguments

# Upload your dictionary file (CSV format) in Colab
uploaded = files.upload()

# Read the uploaded dictionary file (assuming it's in CSV format)
filename = list(uploaded.keys())[0]  # Get the file name
dictionary = pd.read_csv(filename)

# Check the first few rows of the dictionary
dictionary.head()

# Initialize the Llama tokenizer (replace "facebook/llama-3-7b" with your model)
tokenizer = LlamaTokenizer.from_pretrained("facebook/layerskip-llama2-7B")

# Preprocess the data - tokenize the dictionary entries
def tokenize_pair(row):
    english_text = row['english']  # Assuming the column name is 'English'
    kriol_text = row['kriol']     # Assuming the column name is 'Kriol'

    return tokenizer(english_text, kriol_text, padding="max_length", truncation=True)

# Apply tokenization
tokenized_dict = dictionary.apply(tokenize_pair, axis=1)

# Convert the tokenized data to a Hugging Face Dataset object
dataset = Dataset.from_pandas(tokenized_dict)

# Split the dataset into train and validation sets
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
val_dataset = dataset['test']

# Load the pre-trained Llama model (use the correct model version)
model = LlamaForCausalLM.from_pretrained("facebook/layerskip-llama2-7B")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",             # output directory
    evaluation_strategy="epoch",        # evaluation strategy
    learning_rate=2e-5,                 # learning rate
    per_device_train_batch_size=4,      # train batch size
    per_device_eval_batch_size=4,       # eval batch size
    weight_decay=0.01,                  # weight decay
    logging_dir="./logs",               # logging directory
    logging_steps=10,                   # log every 10 steps
    save_steps=500,                     # save checkpoint every 500 steps
    num_train_epochs=3,                 # number of epochs
)

# Initialize the Trainer with the model, arguments, and datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Start training
trainer.train()

# Evaluate the model after training
trainer.evaluate()

# Save the trained model and tokenizer
model.save_pretrained("./final_model")
tokenizer.save_pretrained("./final_model")


Saving dictionary.csv to dictionary (1).csv
Saving english_kriol_dataset.csv to english_kriol_dataset (1).csv


KeyError: 'English'